i regularly check developers journal, so you can leave your comments or what function you wanna see in later versions here.


test\_v.0.0.1 \-\- rough SIR model only introduced 3 statuses of cell

neighbours\_status\_counter has been introduced for  s\-e converting function

operating\_array been introduced to solve "d" problem caused by neighbours\_status\_counter

2\-dimensional input be converted into 2\-dimensional operating array, and release a 2\-dimensional output

only one grid has been used from start to end during plotting, changes over time has not been memorized



In [0]:
import numpy as np
import matplotlib.pyplot as plt

susceptible= 0
infected= 1
recovered= 2

def neighbours_status_counter(x, i, j, target_status):
    
    y=np.sum((x[i-1:i+2,j-1:j+2] == target_status))-np.sum(x[i,j]==target_status)
    return y
    


def advance(input_array, infect_factor, recover_factor):
    
    
    def operating_array_convertor(input_array):
    
        i_n,i_m=np.shape(input_array)
    
        operating_array=np.zeros((i_n+2,i_m+2))
    
        for i in range (i_n):
            for j in range (i_m):
                operating_array[1+i,1+j]=input_array[i,j]         
        return operating_array
    
    
    operating_array=operating_array_convertor(input_array)
    o_n,o_m=np.shape(operating_array)
    operating_result = np.zeros((o_n, o_m))
    for i in range(1, o_n-1):
        for j in range(1, o_m-1):
            
            if operating_array[i,j] == 0:
                if neighbours_status_counter(operating_array, i, j, 1) > 0:
                    if np.random.rand() < infect_factor:
                        operating_result[i,j] = 1
                    else:
                        operating_result[i,j] = 0
                else:
                    operating_result[i,j] = 0

            if operating_array[i,j] == 1:
                if np.random.rand() < recover_factor:
                    operating_result[i,j] = 2
                else:
                    operating_result[i,j] = 1
            if operating_array[i,j] == 2:
                    operating_result[i,j] = 2
                    
    i_n,i_m=np.shape(input_array)
    result=np.zeros((i_n,i_m))
    for i in range(i_n):
        for j in range(i_m):
            result[i,j]=operating_result[1+i,j+1]
    
    return result


grid = np.zeros((5,5))
grid[2,2] = 1
print(grid)


fig, axes = plt.subplots(7, 7, figsize=(14,14))
for i in range(7):
    for j in range(7):
        
        axes[i][j].imshow(grid, vmax=2,vmin=0)
        axes[i][j].axis("off")
        grid= advance(grid,0.25,0.25)

test\_v.0.1.0 \-\- curve plotting function is now available, colours used to represent different status of cells has been modified to moodle version

to plot change of population over time, grid changes has to be memorized, thus grid\_change\_over\_time function has been introduced, this function allow 2\-dimensional output to be collected into a 3\-dimensional array in order of time

2 functions has been introduced to cut sections and extract the number of cells at certain status form the section from 3 dimensional array, and put the certain status number into an array among time

hexadecimal colouring code was extracted from moodle directly



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

susceptible= 0
infected= 1
recovered= 2
infect_factor=0.25
recover_factor=0.25

def cells_status_counter(input_array,target_status):
    
    y=np.sum(input_array[:,:] == target_status)
    return y

def neighbours_status_counter(input_array, i, j, target_status):
    
    y=np.sum((input_array[i-1:i+2,j-1:j+2] == target_status))-np.sum(input_array[i,j]==target_status)
    return y
    

def advance(input_array):
    
    def operating_array_convertor(input_array):
    
        i_n,i_m=np.shape(input_array)
    
        operating_array=np.zeros((i_n+2,i_m+2))
    
        for i in range (i_n):
            for j in range (i_m):
                operating_array[1+i,1+j]=input_array[i,j]         
        return operating_array
    
    
    operating_array=operating_array_convertor(input_array)
    o_n,o_m=np.shape(operating_array)
    operating_result = np.zeros((o_n, o_m))
    for i in range(1, o_n-1):
        for j in range(1, o_m-1):
            
            if operating_array[i,j] == 0:
                if neighbours_status_counter(operating_array, i, j, 1) > 0:
                    if np.random.rand() < infect_factor:
                        operating_result[i,j] = 1
                    else:
                        operating_result[i,j] = 0
                else:
                    operating_result[i,j] = 0

            if operating_array[i,j] == 1:
                if np.random.rand() < recover_factor:
                    operating_result[i,j] = 2
                else:
                    operating_result[i,j] = 1
            if operating_array[i,j] == 2:
                    operating_result[i,j] = 2
                    
    i_n,i_m=np.shape(input_array)
    result=np.zeros((i_n,i_m))
    for i in range(i_n):
        for j in range(i_m):
            result[i,j]=operating_result[1+i,j+1]
    
    return result


def grid_change_over_time(input_array,timesteps):
    i_n,i_m=np.shape(input_array)
    result=np.zeros((i_n,i_m,timesteps))
    
    for i in range (timesteps):
        result[:,:,i]=input_array
        input_array= advance(input_array)
    return result



grid = np.zeros((16,16))
grid[2,2] = 1
print(grid)

operating_grid=grid

rows_number=7
columns_number=7


def target_status_array_overtime(grid_change_over_time,target_status):
    n,m,t=np.shape(grid_change_over_time)
        
    result=np.zeros(t)
    
    for i in range (t):
        result[i]=cells_status_counter(grid_change_over_time[:,:,i],target_status)
    return result


fig, axes = plt.subplots(rows_number, columns_number, figsize=(14,14))
cmap = mcolors.ListedColormap(['#393BA6', '#E0BD63', '#D4A0DD'])
bounds = [0,0.5,1.5, 2]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

grid_change_over_time=grid_change_over_time(operating_grid,rows_number*columns_number)

for i in range(rows_number):
   
    for j in range(columns_number):

        axes[i][j].imshow(grid_change_over_time[:,:,j+i*columns_number], cmap= cmap, norm= norm)
        axes[i][j].axis("off")

plt.show()
print(target_status_array_overtime(grid_change_over_time,0))
print(target_status_array_overtime(grid_change_over_time,1))
print(target_status_array_overtime(grid_change_over_time,2))

plt.figure(figsize=(3,2))
plt.plot(target_status_array_overtime(grid_change_over_time,0),color='#393BA6')
plt.plot(target_status_array_overtime(grid_change_over_time,1),color='#E0BD63')
plt.plot(target_status_array_overtime(grid_change_over_time,2),color='#D4A0DD')
plt.xlabel("day")
n,m=np.shape(operating_grid)
#plt.xticks(np.arange(0,rows_number*columns_number,1))
#plt.yticks(np.arange(0,n*m,1))
plt.ylabel("number of person")
plt.title("")
plt.legend(["susceptible","infected","recovered"])

additional function: initial grid generator

as it is almost impossible for human\-being to introduce a 2\-dimensional array with size over 100x100, a small function has been designed

this function is also abled to control population density and infector density, which would be useful to generate empty cells in later SEIQR full model



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

empty_cell= 0
susceptible= S= 1
exposed= E= 2
infected= I= 3
quarantined= Q= 4
removed= R= 5

def initial_grid_generator(n,m,D,I):
    
    y=np.zeros((n,m))
    N=n*m
    for i in range (n):
        for j in range (m):
            if np.random.rand()<D:
                if np.random.rand()<I:
                    y[i,j]= infected
                else:
                    y[i,j]= susceptible
            else:
                y[i,j]= 0
                
    return y

cmap = mcolors.ListedColormap(["#000000", '#001DFF', "#00FF00", '#FF0000', "#00FFF0", '#F0FF00'])
bounds = [0, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)
plt.imshow(initial_grid_generator(100,100,0.4,0.004), cmap= cmap, norm= norm)

test\_v.0.1.1 \-\- initial grid generator has been introduced into test\_v.0.1.0

the introduction of initial grid generator proves that test\_v.0.1.0 was abled to treat with big array with size over 100x100, the problem is that the initial grid generator was designed for generating empty cells as well, test\_v.0.1.0 is an SIR model without empty cell concept, it cannot fully demonstrate the ability of the grid generator

#

notes written by Ellen T. , thank you Ellen



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

susceptible= 0
infected= 1
recovered= 2
infect_factor=0.25
recover_factor=0.25

def initial_grid_generator(n,m,D,I):
    
    y=np.zeros((n,m))
    N=n*m
    for i in range (n):
        for j in range (m):
            if np.random.rand()<D:
                if np.random.rand()<I:
                    y[i,j]= infected
                else:
                    y[i,j]= susceptible
            else:
                y[i,j]= 0
                
    return y

# this function counts how many cells in input_array have a value equal to target_status 
# e.g. 0 for susceptible, 1 for infected
def cells_status_counter(input_array,target_status):
    
    y=np.sum(input_array[:,:] == target_status)
    return y

# This function counts how many neighboring cells of a given cell (i, j) have a specific target_status
# counts only the neighbors, excluding the center cell (i, j), that match target_status
def neighbours_status_counter(input_array, i, j, target_status):
    
    y=np.sum((input_array[i-1:i+2,j-1:j+2] == target_status))-np.sum(input_array[i,j]==target_status)
    return y
    
# This function is used to advance the state of a grid-based simulation
def advance(input_array):
    
    # Helper function
    # pads input_array with zeros
    def operating_array_convertor(input_array):
    
        i_n,i_m=np.shape(input_array)
    
        operating_array=np.zeros((i_n+2,i_m+2))
    
        for i in range (i_n):
            for j in range (i_m):
                operating_array[1+i,1+j]=input_array[i,j]         
        return operating_array
    
    
    operating_array=operating_array_convertor(input_array)
    o_n,o_m=np.shape(operating_array)
    # A new grid to store the updated state after processing
    operating_result = np.zeros((o_n, o_m))
    # Loop through internal cells excluding borders
    for i in range(1, o_n-1):
        for j in range(1, o_m-1):
            
            if operating_array[i,j] == 0:
                if neighbours_status_counter(operating_array, i, j, 1) > 0:
                    if np.random.rand() < infect_factor:
                        operating_result[i,j] = 1
                    else:
                        operating_result[i,j] = 0
                else:
                    operating_result[i,j] = 0

            if operating_array[i,j] == 1:
                if np.random.rand() < recover_factor:
                    operating_result[i,j] = 2
                else:
                    operating_result[i,j] = 1
            if operating_array[i,j] == 2:
                    operating_result[i,j] = 2
    
    # convert back to original grid size
    i_n,i_m=np.shape(input_array)
    result=np.zeros((i_n,i_m))
    for i in range(i_n):
        for j in range(i_m):
            result[i,j]=operating_result[1+i,j+1]
    
    return result


# This function simulates how the state of the grid evolves over a given number of timesteps
def grid_change_over_time(input_array,timesteps):
    i_n,i_m=np.shape(input_array)
    result=np.zeros((i_n,i_m,timesteps))
    
    for i in range (timesteps):
        result[:,:,i]=input_array # Store the current state of the grid in the result array
        input_array= advance(input_array)
    return result



grid = initial_grid_generator(100,100,0.6,0.01)

print(grid)

operating_grid=grid

rows_number=7
columns_number=7


def target_status_array_overtime(grid_change_over_time,target_status):
    n,m,t=np.shape(grid_change_over_time)
        
    result=np.zeros(t)
    
    for i in range (t):
        result[i]=cells_status_counter(grid_change_over_time[:,:,i],target_status)
    return result


fig, axes = plt.subplots(rows_number, columns_number, figsize=(14,14))
cmap = mcolors.ListedColormap(['#393BA6', '#E0BD63', '#D4A0DD'])
bounds = [0,0.5,1.5, 2]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

grid_change_over_time=grid_change_over_time(operating_grid,rows_number*columns_number)

for i in range(rows_number):
   
    for j in range(columns_number):

        axes[i][j].imshow(grid_change_over_time[:,:,j+i*columns_number], cmap= cmap, norm= norm)
        axes[i][j].axis("off")

plt.show()
print(target_status_array_overtime(grid_change_over_time,0))
print(target_status_array_overtime(grid_change_over_time,1))
print(target_status_array_overtime(grid_change_over_time,2))

plt.figure(figsize=(3,2))
plt.plot(target_status_array_overtime(grid_change_over_time,0),color='#393BA6')
plt.plot(target_status_array_overtime(grid_change_over_time,1),color='#E0BD63')
plt.plot(target_status_array_overtime(grid_change_over_time,2),color='#D4A0DD')

test\_v.0.2.0 \-\- draft for SEIQR full model with empty cell concept, and other new parameters: d\_max, probabilities and time\_last for each conversion

brand new status\-number indicating system, using 0,1,2,3,4,5 to replace 0,1,2, where 0 here now is indicating empty cell but not susceptible cells

to introduce the time\_last concept, the final time of conversion in each cell should be recorded, so the whole time line should be memorized together, expansion of the old 2\-dimensional operating array to 4\-dimensioanl is necessary: one for the timeline, one for the recording of time\_last at each cell location, the advance function was fused with grid\_change\_oovertime function for the same reason, the final output of this function is still 3\-dimensional as the old grid\_change\_time function

operating\_array\_convertor function has been cancelled, the operating array would be converted directly

to fit with the expansion of operating array, the input array for the neighbour status counter function was also expanded to 4\-d

colours representing different cell status has now been reset

graph plotting system has been expanded

d\_max and d are separated , this is for the further updating, makes the introducing of time lockdown\(d decrease after certain time\) to be possible



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

empty_cell= 0
susceptible= S= 1
exposed= E= 2
infected= I= 3
quarantined= Q= 4
removed= R= 5

sub_p_e_n= 0.15
p_i= 0.6
p_q= 0.5
p_r= 0.5

d_max= 10
d=2

t_e= 0
t_i= 5
t_q= 7
t_r= 7

infect_factor=0.25
recover_factor=0.25

def initial_grid_generator(n,m,D,I):
    
    y=np.zeros((n,m))
    N=n*m
    for i in range (n):
        for j in range (m):
            if np.random.rand()<D:
                if np.random.rand()<I:
                    y[i,j]= infected
                else:
                    y[i,j]= susceptible
            else:
                y[i,j]= 0
                
    return y

def cells_status_counter(input_array,target_status):
    
    y=np.sum(input_array[:,:] == target_status)
    return y

def neighbours_status_counter(input_array, i, j, target_status, d, t):
    
    y=np.sum((input_array[i-d:i+d+1,j-d:j+d+1,t,0] == target_status))-np.sum(input_array[i,j,t,0]==target_status)
    return y

    
def grid_change_over_time(input_array,timesteps):
    
    i_n,i_m=np.shape(input_array)
    operating_array= np.zeros((i_n+2*d_max,i_m+2*d_max,timesteps,2))
    for i in range (i_n):
        for j in range (i_m):
            operating_array[d_max+i,d_max+j,0,0]=input_array[i,j]

    
    for t in range (timesteps-1):
        for i in range(d_max, i_n+d_max):
            for j in range(d_max, i_m+d_max):
                
                if operating_array[i,j,t,0] == 0:
                    operating_array[i,j,t+1,0] = 0
                    operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                    
                if operating_array[i,j,t,0] == 1:
                    if operating_array[i,j,t,1] <= t-t_e and np.random.rand() <= sub_p_e_n*(neighbours_status_counter(operating_array, i, j, 2, d, t)+ neighbours_status_counter(operating_array,i,j,3,d,t)):
                        operating_array[i,j,t+1,0] = 2
                        operating_array[i,j,t+1,1] = t+1
                    else:
                        operating_array[i,j,t+1,0] = 1
                        operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                            
                if operating_array[i,j,t,0] == 2:
                    if operating_array[i,j,t,1] <= t-t_i and np.random.rand() <= p_i:
                        operating_array[i,j,t+1,0] = 3
                        operating_array[i,j,t+1,1] = t+1
                    else:
                        operating_array[i,j,t+1,0] = 2
                        operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                        
                        
                if operating_array[i,j,t,0] == 3:
                    if operating_array[i,j,t,1] <= t-t_r and np.random.rand() <= p_r:
                        operating_array[i,j,t+1,0] = 5
                        operating_array[i,j,t+1,1] = t+1
                    elif operating_array[i,j,t,1] <= t-t_q and np.random.rand() <= p_q:
                        operating_array[i,j,t+1,0] = 4
                        operating_array[i,j,t+1,1] = t+1
                    else:
                        operating_array[i,j,t+1,0] = 3
                        operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                            
                if operating_array[i,j,t,0] == 4:
                    if operating_array[i,j,t,1] <= t-t_r and np.random.rand() <= p_r:
                        operating_array[i,j,t+1,0] = 5
                        operating_array[i,j,t+1,1] = t+1
                    else:
                        operating_array[i,j,t+1,0] = 4
                        operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                        
                if operating_array[i,j,t,0] == 5:
                    operating_array[i,j,t+1,0] = 5
                    operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                    
                    
                    
    i_n,i_m=np.shape(input_array)
    result=np.zeros((i_n,i_m,timesteps))
    for tt in range(timesteps):
        for ii in range(i_n):
            for jj in range(i_m):
                result[ii,jj,tt]=operating_array[ii+d_max,jj+d_max,tt,0]
    
    return result





grid = initial_grid_generator(140,140,0.4,0.004)
print(grid)

operating_grid=grid

rows_number=3
columns_number=2
step=10
relative_size=6


grid_change_over_time=grid_change_over_time(operating_grid,rows_number*columns_number*step)








def target_status_array_overtime(grid_change_over_time,target_status,t):
    n,m,t=np.shape(grid_change_over_time)
        
    result=np.zeros(t)
    
    for i in range (t):
        result[i]=cells_status_counter(grid_change_over_time[:,:,i],target_status)
    return result


fig, axes = plt.subplots(rows_number, columns_number, figsize=(columns_number*relative_size,rows_number*relative_size))
cmap = mcolors.ListedColormap(["#000000", '#001DFF', "#00FF00", '#FF0000', "#00FFF0", '#F0FF00'])
bounds = [0, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)


for i in range(rows_number):
   
    for j in range(columns_number):

        axes[i][j].imshow(grid_change_over_time[:,:,(columns_number*i+j)*step], cmap= cmap, norm= norm)
        axes[i][j].axis("off")

plt.show()

plt.figure(figsize=(3,2))
plt.plot(target_status_array_overtime(grid_change_over_time,1,rows_number*columns_number),color="#001DFF")
plt.plot(target_status_array_overtime(grid_change_over_time,2,rows_number*columns_number),color="#00FF00")
plt.plot(target_status_array_overtime(grid_change_over_time,3,rows_number*columns_number),color="#FF0000")
plt.plot(target_status_array_overtime(grid_change_over_time,4,rows_number*columns_number),color="#00FFF0")
plt.plot(target_status_array_overtime(grid_change_over_time,5,rows_number*columns_number),color="#F0FF00")
plt.xlabel("day")
n,m=np.shape(operating_grid)
#plt.xticks(np.arange(0,rows_number*columns_number,1))
#plt.yticks(np.arange(0,n*m,1))
plt.ylabel("number of person")
plt.title("")
plt.legend(["susceptible", "exposed", "infected", "quarantined", "recovered"])

model\_v.1.0.0 \-\- first version of released model

the whole code has been separated into 3, which makes sure the same grid is used during simulation

fully noted

parameters has been highlighted

plotting system became more user friendly

repaired known errors


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

#this cell generate a grid with know size, shape, population density, and initial infector density(you can also add up an initial exposed population density among all population if you want), and the grid you generated would be shown below this cell if it is working normally, so that you can check whether you like the grid all not, and you are able to edit this grid by changing parameters which are circled by "#"s.

#this cell is seperated from the mainbody and must be runned before all the others, as hopefully the grid you are going to use for later analysis(effect of change in p, d, t etc. ) should be identicle(unless you wanna check the effect of the grid parameters such as population density of initial infectors property etc., then you would better to generate and save grids with different parameters seperately, i will put a ready_code below).

#grid_1 = initial_grid_generator(n,m,D_1,I_1)
#grid_2 = initial_grid_generator(n,m,D_2,I_2)
#grid_3 = initial_grid_generator(n,m,D_3,I_3)

#if you have generated several starter grids, later operating code should be expanded in a similar way, basically saveing operating results seperately is a good habit as you dont know where they would be needed.


empty_cell= 0
susceptible= S= 1
exposed= E= 2
infected= I= 3
quarantined= Q= 4
removed= R= 5

def initial_grid_generator(n,m,D,I):
    
    y=np.zeros((n,m))
    for i in range (n):
        for j in range (m):
            if np.random.rand()<D:
                if np.random.rand()<I:
                    y[i,j]= infected
                else:
                    y[i,j]= susceptible
            else:
                y[i,j]= 0
                
    return y
#######################################################
                                                      #
grid = initial_grid_generator(140,140,0.2,0.004)      #
                                                      #
#######################################################
cmap = mcolors.ListedColormap(["#000000", '#001DFF', "#00FF00", '#FF0000', "#00FFF0", '#F0FF00'])
bounds = [0, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)
print(grid)
plt.imshow(grid, cmap= cmap, norm= norm)

In [0]:

#this function is used to count the number of neibouring cells in certain status "target_status", in a diameter "d", at certain location input_array[i,j] at time "t".
#just a small tip: the input_array here is 4-dimensional, including location, time, and a time_last memory dimension, as it was designed to be used in another function, if you wanna use it seperately, you have to modify it manually.
def neighbours_status_counter(input_array, i, j, target_status, d, t):
    
    y=np.sum((input_array[i-d:i+d+1,j-d:j+d+1,t,0] == target_status))-np.sum(input_array[i,j,t,0]==target_status)
    return y


#mainbody with huuuuuuuge if statements, i know it doesnt look gorgeous and i was trying to write a common function which can automatically run those statements, clearly i failed, maybe this autofunction would be introduced in later versions, maybe not, we ll see...

#the input_array here is 2-dimensional,  and timesteps would be defined by the rows and columns and time seperation between each graph you genarate at last automatically, of course you can change it manualy as you like if you wanna have a dataset with known timesteps(days here).
#the output here would be a 3-dimensional array with same size as the input_array, and the extra dimension is the timeline with length of timesteps, this allows you to catch the data and cut a section at any certain timestep you want during the timeline by [:,:,t].
def grid_change_over_time(input_array,timesteps):
    
    #due to the "d", cells at edge would not be allowed to use the neighbour counting function, so we create a new array with bigger size and insert all data from input_array to the centre of it.
    #the 2-dimensional array was also expanded to 4-dimensional array here, plz note that.
    i_n,i_m=np.shape(input_array)
    operating_array= np.zeros((i_n+2*d_max,i_m+2*d_max,timesteps,2))
    for i in range (i_n):
        for j in range (i_m):
            operating_array[d_max+i,d_max+j,0,0]=input_array[i,j]

    #if statements, you can create certain type of conversion by looking at the common structure of those statements, dont forget to introduce new "p" and "t".
    for t in range (timesteps-1):
        for i in range(d_max, i_n+d_max):
            for j in range(d_max, i_m+d_max):
                
                if operating_array[i,j,t,0] == 0:
                    operating_array[i,j,t+1,0] = 0
                    operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                    
                if operating_array[i,j,t,0] == 1:
                    if operating_array[i,j,t,1] <= t-t_e and np.random.rand() <= sub_p_e_n*(neighbours_status_counter(operating_array, i, j, 2, d, t)+ neighbours_status_counter(operating_array,i,j,3,d,t)):
                        operating_array[i,j,t+1,0] = 2
                        operating_array[i,j,t+1,1] = t+1
                    else:
                        operating_array[i,j,t+1,0] = 1
                        operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                            
                if operating_array[i,j,t,0] == 2:
                    if operating_array[i,j,t,1] <= t-t_i and np.random.rand() <= p_i:
                        operating_array[i,j,t+1,0] = 3
                        operating_array[i,j,t+1,1] = t+1
                    else:
                        operating_array[i,j,t+1,0] = 2
                        operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                        
                        
                if operating_array[i,j,t,0] == 3:
                    if operating_array[i,j,t,1] <= t-t_r and np.random.rand() <= p_r:
                        operating_array[i,j,t+1,0] = 5
                        operating_array[i,j,t+1,1] = t+1
                    elif operating_array[i,j,t,1] <= t-t_q and np.random.rand() <= p_q:
                        operating_array[i,j,t+1,0] = 4
                        operating_array[i,j,t+1,1] = t+1
                    else:
                        operating_array[i,j,t+1,0] = 3
                        operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                            
                if operating_array[i,j,t,0] == 4:
                    if operating_array[i,j,t,1] <= t-t_r and np.random.rand() <= p_r:
                        operating_array[i,j,t+1,0] = 5
                        operating_array[i,j,t+1,1] = t+1
                    else:
                        operating_array[i,j,t+1,0] = 4
                        operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                        
                if operating_array[i,j,t,0] == 5:
                    operating_array[i,j,t+1,0] = 5
                    operating_array[i,j,t+1,1] = operating_array[i,j,t,1]
                    
    #the result here is 3-dimensional as i ve just mentioned befor, the forth dimension for time_last memory is gived up here as it is totally useless for plotting gragh or analysis(you set up those numbers!)
    result=np.zeros((i_n,i_m,timesteps))
    for tt in range(timesteps):
        for ii in range(i_n):
            for jj in range(i_m):
                result[ii,jj,tt]=operating_array[ii+d_max,jj+d_max,tt,0]
    
    return result


###################################################################################
                                                                                  #
#here is the area to modify parameters and the final graph.                       #
                                                                                  #
                                                                                  #
rows_number=6                                                                     #
columns_number=4                                                                  #
step=5                                                                            #
relative_size=5                                                                   #
                                                                                  #
sub_p_e_n= 0.15                                                                   #
p_i= 0.6                                                                          #
p_q= 0.5                                                                          #
p_r= 0.5                                                                          #
                                                                                  #
d_max= 5                                                                          #
d=4                                                                               #
                                                                                  #
t_e= 0                                                                            #
t_i= 7                                                                            #
t_q= 5                                                                            #
t_r= 21                                                                           #
                                                                                  #
                                                                                  #
                                                                                  #
                                                                                  #
                                                                                  #
###################################################################################

#times would be needed if you used too long timelength or grid with too big size.


fig, axes = plt.subplots(rows_number, columns_number, figsize=(columns_number*relative_size,rows_number*relative_size))

grid_change_over_time=grid_change_over_time(grid,rows_number*columns_number*step)

for i in range(rows_number):
   
    for j in range(columns_number):

        axes[i][j].imshow(grid_change_over_time[:,:,(columns_number*i+j)*step], cmap= cmap, norm= norm)
        axes[i][j].axis("off")

plt.show()

In [0]:
def cells_status_counter(input_array,target_status):
    
    y=np.sum(input_array[:,:] == target_status)
    return y

def target_status_array_overtime(grid_change_over_time,target_status,t):
    n,m,t=np.shape(grid_change_over_time)
        
    result=np.zeros(t)
    
    for i in range (t):
        result[i]=cells_status_counter(grid_change_over_time[:,:,i],target_status)
    return result

# 0 for auto mode, and the x axis would be the full size.
# 1 for manual mode, you have to choose the length of x-axis, sometimes it would help if you have got a too long timelength.
#########################
                        #
mode_choosing = 0       #
x_size = 7              #
y_size = 5              #
                        #
#########################

##########
         #
t= 200   # the length of the x-axis you are plotting(only work in manual mode)
         #
##########



if mode_choosing == 0:
    plt.figure(figsize=(x_size,y_size))
    plt.plot(target_status_array_overtime(grid_change_over_time,1,rows_number*columns_number*step),color="#001DFF")
    plt.plot(target_status_array_overtime(grid_change_over_time,2,rows_number*columns_number*step),color="#00FF00")
    plt.plot(target_status_array_overtime(grid_change_over_time,3,rows_number*columns_number*step),color="#FF0000")
    plt.plot(target_status_array_overtime(grid_change_over_time,4,rows_number*columns_number*step),color="#00FFF0")
    plt.plot(target_status_array_overtime(grid_change_over_time,5,rows_number*columns_number*step),color="#F0FF00")
    plt.xlabel("day")
    plt.ylabel("number of person")
    plt.title("")
    plt.legend(["susceptible", "exposed", "infected", "quarantined", "removed"])
if mode_choosing == 1:
    if t > rows_number*columns_number*step:
        print("The time you chose for x-axis was too long, choose a smaller number or change to the automatic mode. --LL")
    else:
        plt.figure(figsize=(x_size,y_size))
        plt.plot(target_status_array_overtime(grid_change_over_time,1,t),color="#001DFF")
        plt.plot(target_status_array_overtime(grid_change_over_time,2,t),color="#00FF00")
        plt.plot(target_status_array_overtime(grid_change_over_time,3,t),color="#FF0000")
        plt.plot(target_status_array_overtime(grid_change_over_time,4,t),color="#00FFF0")
        plt.plot(target_status_array_overtime(grid_change_over_time,5,t),color="#F0FF00")
        plt.xlabel("day")
        plt.ylabel("number of person")
        plt.title("")
        plt.legend(["susceptible", "exposed", "infected", "quarantined", "removed"])

test\_v.0.2.1 \-\- automatic cell convertor introduced

noticed that there are tons of meaningless repeating if statements for each cell status in the main function, and it also makes the possible future addition of new converting pathways became a nightmare, the introduction of new general cell convertor and general converting parameters recording array have been put on my to do list since test\_v.0.2.0. now it finally came true

codes put here just for recording, further simplify and adjust still needed, but you are also welcomed to test it before final release, plz noticed this test\_v.0.2.1 is generally based on test\_v.0.2.0 codes


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

empty_cell= 0
susceptible= S= 1
exposed= E= 2
infected= I= 3
quarantined= Q= 4
removed= R= 5

sub_p_s_e=0.15

p_e_i=0.9
p_i_q=0.5
p_i_r=0.5
p_q_r=0.5

d_max= 10
d=1



def initial_grid_generator(n,m,D,I):
    
    y=np.zeros((n,m))
    N=n*m
    for i in range (n):
        for j in range (m):
            if np.random.rand()<D:
                if np.random.rand()<I:
                    y[i,j]= infected
                else:
                    y[i,j]= susceptible
            else:
                y[i,j]= 0
                
    return y

def cells_status_counter(input_array,target_status):
    
    y=np.sum(input_array[:,:] == target_status)
    return y

def neighbours_status_counter(input_array, i, j, target_status,t, d):
    
    y=np.sum((input_array[i-d:i+d+1,j-d:j+d+1,t,0] == target_status))-np.sum(input_array[i,j,t,0]==target_status)
    return y

def grid_change_over_time(input_array,timesteps):

    i_n,i_m=np.shape(input_array)
    operating_array= np.zeros((i_n+2*d_max,i_m+2*d_max,timesteps,2))
    operating_array[d_max:d_max+i_n,d_max:d_max+i_m,0,0]=input_array[:,:]
    
    
    def cell_A_to_B_convertor(i,j,A,B):
        
        A_cell_at_t_plus_one=np.zeros(2)
        
        def p_s_e(i,j,sub_p_s_e,d):
            p_e= sub_p_s_e* (neighbours_status_counter(operating_array,i,j,2,t,d)+ neighbours_status_counter(operating_array,i,j,3,t,d))
            return p_e
        


        converting_probability_array=np.array([[0   ,0   ,0   ,0   ,0   ,0   ],
                                          [0   ,0   ,p_s_e(i,j,0.8,d),0   ,0   ,0   ],
                                          [0   ,0   ,0   ,0.9 ,0   ,0   ],
                                          [0   ,0   ,0   ,0   ,0.5 ,0.5 ],
                                          [0   ,0   ,0   ,0   ,0   ,0.5 ],
                                          [0   ,0   ,0   ,0   ,0   ,0   ]])

        transitional_delay_array=np.array([[0 ,0 ,0 ,0 ,0 ,0 ],
                                      [0 ,0 ,0 ,0 ,0 ,0 ],
                                      [0 ,0 ,0 ,5 ,0 ,0 ],
                                      [0 ,0 ,0 ,0 ,1 ,5 ],
                                      [0 ,0 ,0 ,0 ,0 ,5 ],
                                      [0 ,0 ,0 ,0 ,0 ,0 ]])
                
        if operating_array[i,j,t,1] <= t-transitional_delay_array[A,B] and np.random.rand() <= converting_probability_array[A,B]:
            A_cell_at_t_plus_one[0] = B
            A_cell_at_t_plus_one[1] = t+1
        else:
            A_cell_at_t_plus_one[0] = A
            A_cell_at_t_plus_one[1] = operating_array[i,j,t,1]
                    
                    
        return A_cell_at_t_plus_one
    
    
    for t in range (timesteps-1):
        for i in range(d_max, i_n+d_max):
            for j in range(d_max, i_m+d_max):
                
                if operating_array[i,j,t,0] == 0:
                    operating_array[i,j,t+1,:] = 0
                    
                if operating_array[i,j,t,0] == 1:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,1,2)
                            
                if operating_array[i,j,t,0] == 2:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,2,3)
                
                if operating_array[i,j,t,0] == 3:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,5)
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,4)

                if operating_array[i,j,t,0] == 4:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,4,5)
                        
                if operating_array[i,j,t,0] == 5:
                    operating_array[i,j,t+1,:] = operating_array[i,j,t,:]
                    
    result=np.zeros((i_n,i_m,timesteps))
    for tt in range(timesteps):
        for ii in range(i_n):
            for jj in range(i_m):
                result[ii,jj,tt]=operating_array[ii+d_max,jj+d_max,tt,0]
    
    return result





grid = initial_grid_generator(100,100,0.4,0.05)
print(grid)

operating_grid=grid

rows_number=20
columns_number=4



grid_change_over_time=grid_change_over_time(operating_grid,rows_number*columns_number)








def target_status_array_overtime(grid_change_over_time,target_status,t):
    n,m,t=np.shape(grid_change_over_time)
        
    result=np.zeros(t)
    
    for i in range (t):
        result[i]=cells_status_counter(grid_change_over_time[:,:,i],target_status)
    return result


fig, axes = plt.subplots(rows_number, columns_number, figsize=(columns_number*5,rows_number*5))
cmap = mcolors.ListedColormap(["#000000", "#001DFF", "#00FF00", "#FF0000", "#00FFF0", "#F0FF00"])
bounds = [0, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)


for i in range(rows_number):
   
    for j in range(columns_number):

        axes[i][j].imshow(grid_change_over_time[:,:,columns_number*i+j], cmap= cmap, norm= norm)
        axes[i][j].axis("off")

plt.show()

plt.figure(figsize=(3,2))
plt.plot(target_status_array_overtime(grid_change_over_time,1,rows_number*columns_number),color="#001DFF")
plt.plot(target_status_array_overtime(grid_change_over_time,2,rows_number*columns_number),color="#00FF00")
plt.plot(target_status_array_overtime(grid_change_over_time,3,rows_number*columns_number),color="#FF0000")
plt.plot(target_status_array_overtime(grid_change_over_time,4,rows_number*columns_number),color="#00FFF0")
plt.plot(target_status_array_overtime(grid_change_over_time,5,rows_number*columns_number),color="#F0FF00")
plt.xlabel("day")
n,m=np.shape(operating_grid)
plt.ylabel("number of person")
plt.title("")
plt.legend(["susceptible", "exposed", "infected", "quarantined", "removed"])

optional updating for model\_v.1.0.0:

new crucial function allows more flexible research and editing, was going to release as a part of the model\_v.1.1.0, but due to unavoidable def over def over def loops, the function would actually increase the running time compares with old v.1.0.0, so if you just wanna focus on given converting pathways in model\_v.1.0.0, this would be a "downdating". thats why i made this function an optional updating.

you can copy the code below and directly replace the middle section of model\_v.1.0.0, do not forget to modify parameters in arrays



In [0]:
#this function is used to count the number of neibouring cells in certain status "target_status", in a diameter "d", at certain location input_array[i,j] at time "t".
#just a small tip: the input_array here is 4-dimensional, including location, time, and a time_last memory dimension, as it was designed to be used in another function, if you wanna use it seperately, you have to modify it manually.
def neighbours_status_counter(input_array, i, j, target_status, d,t):
    
    y=np.sum((input_array[i-d:i+d+1,j-d:j+d+1,t,0] == target_status))-np.sum(input_array[i,j,t,0]==target_status)
    return y



#the input_array here is 2-dimensional,  and timesteps would be defined by the rows and columns and time seperation between each graph you genarate at last automatically, of course you can change it manualy as you like if you wanna have a dataset with known timesteps(days here).
#the output here would be a 3-dimensional array with same size as the input_array, and the extra dimension is the timeline with length of timesteps, this allows you to catch the data and cut a section at any certain timestep you want during the timeline by [:,:,t].
def grid_change_over_time(input_array,timesteps):

    i_n,i_m=np.shape(input_array)
    operating_array= np.zeros((i_n+2*d_max,i_m+2*d_max,timesteps,2))
    operating_array[d_max:d_max+i_n,d_max:d_max+i_m,0,0]=input_array[:,:]
    
    #welcome to v.1.2.0!the new function here allows you add up new converting pathway by just paste a new row of code and type the status you wanna convert form and to. or remove a pathway by just deleting a single row. dont foget to change to new parameters as well if needed, or they will be zero
    def cell_A_to_B_convertor(i,j,A,B):
        #this would be the output of this function and it is 2-dimensional, one for cell status, the other for time_last memory
        A_cell_at_t_plus_one=np.zeros(2)
        #if i am pathologist i wanna say the concept of proportional infect probability based on neighbour status surrounding you is really genius and so close to the real world, and for a programme developer THIS IS A BULLSHIFT. i dont even know how much extra codes have been writtern and how simple and gorgeous this code would be looks like without those creepy functions, i dont even know how to simplify it
        def p_s_e(i,j,sub_p_s_e,d):
            p_e= sub_p_s_e* (neighbours_status_counter(operating_array,i,j,2,d,t)+ neighbours_status_counter(operating_array,i,j,3,d,t))
            return p_e
        

        #here are the arrays prepared for the automatic convertor. rows number representing the initial status while columns number is the target status
        converting_probability_array=np.array([[0   ,0   ,0   ,0   ,0   ,0   ],
                                               [0   ,0   ,p_s_e(i,j,0.15,d),0   ,0   ,0   ],
                                               [0   ,0   ,0   ,0.3 ,0   ,0   ],
                                               [0   ,0   ,0   ,0   ,0.2 ,0.2 ],
                                               [0   ,0   ,0   ,0   ,0   ,0.4 ],
                                               [0   ,0   ,0   ,0   ,0   ,0   ]])

        transitional_delay_array=np.array([[0 ,0 ,0 ,0 ,0 ,0 ],
                                           [0 ,0 ,0 ,0 ,0 ,0 ],
                                           [0 ,0 ,0 ,5 ,0 ,0 ],
                                           [0 ,0 ,0 ,0 ,1 ,5 ],
                                           [0 ,0 ,0 ,0 ,0 ,5 ],
                                           [0 ,0 ,0 ,0 ,0 ,0 ]]). 
                
        if operating_array[i,j,t,1] <= t-transitional_delay_array[A,B] and np.random.rand() <= converting_probability_array[A,B]:
            A_cell_at_t_plus_one[0] = B
            A_cell_at_t_plus_one[1] = t+1
        else:
            A_cell_at_t_plus_one[0] = A
            A_cell_at_t_plus_one[1] = operating_array[i,j,t,1]
                    
                    
        return A_cell_at_t_plus_one
    
    
    for t in range (timesteps-1):
        for i in range(d_max, i_n+d_max):
            for j in range(d_max, i_m+d_max):
                #if you wanna add some new pathways plz mind the priority of converting
                if operating_array[i,j,t,0] == 0:
                    operating_array[i,j,t+1,:] = 0
                    
                if operating_array[i,j,t,0] == 1:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,1,2)
                            
                if operating_array[i,j,t,0] == 2:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,2,3)
                
                if operating_array[i,j,t,0] == 3:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,5)
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,4)

                if operating_array[i,j,t,0] == 4:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,4,5)
                        
                if operating_array[i,j,t,0] == 5:
                    operating_array[i,j,t+1,:] = operating_array[i,j,t,:]
                    
    result=operating_array[d_max:d_max+i_n,d_max:d_max+i_m,:,0]
    return result


###################################################################################
                                                                                  #
#here is the area to modify parameters and the final graph.                       #
                                                                                  #
                                                                                  #
rows_number=5                                                                     #
columns_number=4                                                                  #
step=5                                                                            #
relative_size=5                                                                   #
                                                                                  #
d_max= 8                                                                          #
d=6                                                                               #
                                                                                  #
                                                                                  #
                                                                                  #
                                                                                  #
                                                                                  #
###################################################################################

#times would be needed if you used too long timelength or grid with too big size.


fig, axes = plt.subplots(rows_number, columns_number, figsize=(columns_number*relative_size,rows_number*relative_size))

grid_change_over_time=grid_change_over_time(grid,rows_number*columns_number*step)

for i in range(rows_number):
   
    for j in range(columns_number):

        axes[i][j].imshow(grid_change_over_time[:,:,(columns_number*i+j)*step], cmap= cmap, norm= norm)
        axes[i][j].axis("off")

plt.show()

model\_v.1.1.0 \-\- new function has been introduced

new function from test\_v.0.2.1 has been introduced to this verison

rearranged codes

repaired some known errors


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def initial_grid_generator(n,m,D,I):
    
    y=np.zeros((n,m))
    for i in range (n):
        for j in range (m):
            if np.random.rand()<D:
                if np.random.rand()<I:
                    y[i,j]= infected
                else:
                    y[i,j]= susceptible
            else:
                y[i,j]= 0
                
    return y

empty_cell = 0
susceptible = S = 1
exposed = E = 2
infected = I = 3
quarantined = Q = 4
removed = R = 5



# n,m determine the size of the grid, D is the population density over all cells, I is the infector density over population

n = 140
m = 140
D = 0.4
I = 0.002



grid = initial_grid_generator(n,m,D,I)
cmap = mcolors.ListedColormap(["#000000", '#001DFF', "#00FF00", '#FF0000', "#00FFF0", '#F0FF00'])
bounds = [0, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)
plt.imshow(grid, cmap= cmap, norm= norm)

In [0]:
def grid_change_over_time(input_array,timesteps):

    i_n,i_m=np.shape(input_array)
    o_n=i_n+2*d_max
    o_m=i_m+2*d_max
    operating_array= np.zeros((o_n, o_m, timesteps, 2))
    operating_array[d_max:d_max+i_n,d_max:d_max+i_m,0,0]=input_array[:,:]
    
    def neighbours_status_counter(input_array, i, j, target_status, d,t):
    
        y=np.sum((input_array[i-d:i+d+1,j-d:j+d+1,t,0] == target_status))-np.sum(input_array[i,j,t,0]==target_status)
        
        return y

    def cell_A_to_B_convertor(i,j,A,B):

        def p_s_e(i,j,sub_p_s_e,d):
            
            p_e= sub_p_s_e* (neighbours_status_counter(operating_array,i,j,2,d,t)+ neighbours_status_counter(operating_array,i,j,3,d,t))
            
            return p_e
        
        A_cell_at_t_plus_one=np.zeros(2)
        converting_probability_array=np.array([[0   ,0   ,0   ,0   ,0   ,0   ],
                                               [0   ,0   ,p_s_e(i,j,0.15,d),0   ,0   ,0   ],
                                               [0   ,0   ,0   ,0.3 ,0   ,0   ],
                                               [0   ,0   ,0   ,0   ,0.2 ,0.2 ],
                                               [0   ,0   ,0   ,0   ,0   ,0.4 ],
                                               [0   ,0   ,0   ,0   ,0   ,0   ]])
        transitional_delay_array=np.array([[0 ,0 ,0 ,0 ,0 ,0 ],
                                           [0 ,0 ,0 ,0 ,0 ,0 ],
                                           [0 ,0 ,0 ,15 ,0 ,0 ],
                                           [0 ,0 ,0 ,0 ,5 ,15 ],
                                           [0 ,0 ,0 ,0 ,0 ,15 ],
                                           [0 ,0 ,0 ,0 ,0 ,0 ]])
                
        if operating_array[i,j,t,1] <= t-transitional_delay_array[A,B] and np.random.rand() <= converting_probability_array[A,B]:
            A_cell_at_t_plus_one[0] = B
            A_cell_at_t_plus_one[1] = t+1
        else:
            A_cell_at_t_plus_one[0] = A
            A_cell_at_t_plus_one[1] = operating_array[i,j,t,1]
        
        return A_cell_at_t_plus_one
    
    
    for t in range (timesteps-1):
        for i in range(d_max, i_n+d_max):
            for j in range(d_max, i_m+d_max):
                if operating_array[i,j,t,0] == 0:
                    operating_array[i,j,t+1,:] = 0
                if operating_array[i,j,t,0] == 1:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,1,2)
                if operating_array[i,j,t,0] == 2:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,2,3)
                if operating_array[i,j,t,0] == 3:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,5)
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,4)
                if operating_array[i,j,t,0] == 4:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,4,5)
                if operating_array[i,j,t,0] == 5:
                    operating_array[i,j,t+1,:] = operating_array[i,j,t,:]
                    
    result = operating_array[d_max:d_max+i_n, d_max:d_max+i_m, :, 0]
    
    return result



rows_number=2
columns_number=4
step=10
relative_size=5

d_max= 8
d=6



fig, axes = plt.subplots(rows_number, columns_number, figsize=(columns_number*relative_size,rows_number*relative_size))
grid_change_over_time=grid_change_over_time(grid,rows_number*columns_number*step)

for i in range(rows_number):
    for j in range(columns_number):
        axes[i][j].imshow(grid_change_over_time[:,:,(columns_number*i+j)*step], cmap= cmap, norm= norm)
        axes[i][j].axis("off")

plt.show()

In [0]:
def cells_status_counter(input_array,target_status):
    
    y=np.sum(input_array[:,:] == target_status)
    
    return y

def target_status_array_overtime(grid_change_over_time,target_status,t):
    
    n,m,t=np.shape(grid_change_over_time)
    result=np.zeros(t)
    
    for i in range (t):
        result[i]=cells_status_counter(grid_change_over_time[:,:,i],target_status)
        
    return result



# 0 for auto mode, and the x axis would be the full size.
# 1 for manual mode, you have to choose the length of x-axis, sometimes it would help if you have got a too long timelength.

mode_choosing = 0
x_size = 7
y_size = 5
t= 200   # the length of the x-axis you are plotting(only work in manual mode)



if mode_choosing == 0:
    plt.figure(figsize=(x_size,y_size))
    plt.plot(target_status_array_overtime(grid_change_over_time,1,rows_number*columns_number*step),color="#001DFF")
    plt.plot(target_status_array_overtime(grid_change_over_time,2,rows_number*columns_number*step),color="#00FF00")
    plt.plot(target_status_array_overtime(grid_change_over_time,3,rows_number*columns_number*step),color="#FF0000")
    plt.plot(target_status_array_overtime(grid_change_over_time,4,rows_number*columns_number*step),color="#00FFF0")
    plt.plot(target_status_array_overtime(grid_change_over_time,5,rows_number*columns_number*step),color="#F0FF00")
    plt.xlabel("day")
    plt.ylabel("number of person")
    plt.title("")
    plt.legend(["susceptible", "exposed", "infected", "quarantined", "removed"])
    
if mode_choosing == 1:
    if t > rows_number*columns_number*step:
        print("The time you chose for x-axis was too long, choose a smaller number or change to the automatic mode. --LL")
    else:
        plt.figure(figsize=(x_size,y_size))
        plt.plot(target_status_array_overtime(grid_change_over_time,1,t),color="#001DFF")
        plt.plot(target_status_array_overtime(grid_change_over_time,2,t),color="#00FF00")
        plt.plot(target_status_array_overtime(grid_change_over_time,3,t),color="#FF0000")
        plt.plot(target_status_array_overtime(grid_change_over_time,4,t),color="#00FFF0")
        plt.plot(target_status_array_overtime(grid_change_over_time,5,t),color="#F0FF00")
        plt.xlabel("day")
        plt.ylabel("number of person")
        plt.title("")
        plt.legend(["susceptible", "exposed", "infected", "quarantined", "removed"])

model\_v.1.1.1 \-\- easier to modify parametres


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def initial_grid_generator(n,m,D,I):
    
    y=np.zeros((n,m))
    for i in range (n):
        for j in range (m):
            if np.random.rand()<D:
                if np.random.rand()<I:
                    y[i,j]= infected
                else:
                    y[i,j]= susceptible
            else:
                y[i,j]= 0
                
    return y

status_array = [0,1,2,3,4,5]
empty_cell = 0
susceptible = S = 1
exposed = E = 2
infected = I = 3
quarantined = Q = 4
removed = R = 5

##############################################

# n,m determine the size of the grid, D is the population density over all cells, I is the infector density over population

n = 140
m = 140
D = 0.4
I = 0.002

##############################################

grid = initial_grid_generator(n,m,D,I)
cmap = mcolors.ListedColormap(["#000000", '#001DFF', "#00FF00", '#FF0000', "#00FFF0", '#F0FF00'])
bounds = [0, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)
plt.imshow(grid, cmap= cmap, norm= norm)

In [0]:
def grid_change_over_time(input_array,timesteps):

    i_n,i_m=np.shape(input_array)
    o_n=i_n+2*d_max
    o_m=i_m+2*d_max
    operating_array= np.zeros((o_n, o_m, timesteps, 2))
    operating_array[d_max:d_max+i_n,d_max:d_max+i_m,0,0]=input_array[:,:]
    
    def neighbours_status_counter(input_array, i, j, target_status, d,t):
    
        y=np.sum((input_array[i-d:i+d+1,j-d:j+d+1,t,0] == target_status))-np.sum(input_array[i,j,t,0]==target_status)
        
        return y

    def cell_A_to_B_convertor(i,j,A,B):

        def p_s_e(i,j,sub_p_s_e,d):
            
            p_e= sub_p_s_e* (neighbours_status_counter(operating_array,i,j,2,d,t)+ neighbours_status_counter(operating_array,i,j,3,d,t))
            
            return p_e
        
        A_cell_at_t_plus_one=np.zeros(2)
        converting_probability_array=np.array([[0   ,0   ,0   ,0   ,0   ,0   ],
                                               [0   ,0   ,p_s_e(i,j,sub_p_s_e,d),0   ,0   ,0   ],
                                               [0   ,0   ,0   ,p_e_i ,0   ,0   ],
                                               [0   ,0   ,0   ,0   ,p_i_q ,p_i_r ],
                                               [0   ,0   ,0   ,0   ,0   ,p_q_r ],
                                               [0   ,0   ,0   ,0   ,0   ,0   ]])
        transitional_delay_array=np.array([[0 ,0 ,0 ,0 ,0 ,0 ],
                                           [0 ,0 ,t_s_e ,0 ,0 ,0 ],
                                           [0 ,0 ,0 ,t_e_i ,0 ,0 ],
                                           [0 ,0 ,0 ,0 ,t_i_q ,t_i_r ],
                                           [0 ,0 ,0 ,0 ,0 ,t_q_r ],
                                           [0 ,0 ,0 ,0 ,0 ,0 ]])
                
        if operating_array[i,j,t,1] <= t-transitional_delay_array[A,B] and np.random.rand() <= converting_probability_array[A,B]:
            A_cell_at_t_plus_one[0] = B
            A_cell_at_t_plus_one[1] = t+1
        else:
            A_cell_at_t_plus_one[0] = A
            A_cell_at_t_plus_one[1] = operating_array[i,j,t,1]
        
        return A_cell_at_t_plus_one
    
    
    for t in range (timesteps-1):
        for i in range(d_max, i_n+d_max):
            for j in range(d_max, i_m+d_max):
                if operating_array[i,j,t,0] == 0:
                    operating_array[i,j,t+1,:] = 0
                if operating_array[i,j,t,0] == 1:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,1,2)
                if operating_array[i,j,t,0] == 2:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,2,3)
                if operating_array[i,j,t,0] == 3:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,5)
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,4)
                if operating_array[i,j,t,0] == 4:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,4,5)
                if operating_array[i,j,t,0] == 5:
                    operating_array[i,j,t+1,:] = operating_array[i,j,t,:]
                    
    result = operating_array[d_max:d_max+i_n, d_max:d_max+i_m, :, 0]
    
    return result

##############################################

d_max= 8
d=6

sub_p_s_e = 0.15
p_e_i = 0.3
p_i_q = 0.2
p_i_r = 0.2
p_q_r = 0.5

t_s_e = 0
t_e_i = 15
t_i_q = 5
t_i_r = 15
t_q_r = 15


rows_number=2
columns_number=4
step=10
relative_size=5

##############################################

fig, axes = plt.subplots(rows_number, columns_number, figsize=(columns_number*relative_size,rows_number*relative_size))
grid_change_over_time=grid_change_over_time(grid,rows_number*columns_number*step)

for i in range(rows_number):
    for j in range(columns_number):
        axes[i][j].imshow(grid_change_over_time[:,:,(columns_number*i+j)*step], cmap= cmap, norm= norm)
        axes[i][j].axis("off")

plt.show()

In [0]:
def cells_status_counter(input_array,target_status):
    
    y=np.sum(input_array[:,:] == target_status)
    
    return y

def target_status_array_overtime(grid_change_over_time,target_status,t):
    
    n,m,t=np.shape(grid_change_over_time)
    result=np.zeros(t)
    
    for i in range (t):
        result[i]=cells_status_counter(grid_change_over_time[:,:,i],target_status)
        
    return result

##############################################

# 0 for auto mode, and the x axis would be the full size.
# 1 for manual mode, you have to choose the length of x-axis, sometimes it would help if you have got a too long timelength.

mode_choosing = 0
x_size = 7
y_size = 5
t= 200   # the length of the x-axis you are plotting(only work in manual mode)

##############################################

if mode_choosing == 0:
    plt.figure(figsize=(x_size,y_size))
    plt.plot(target_status_array_overtime(grid_change_over_time,1,rows_number*columns_number*step),color="#001DFF")
    plt.plot(target_status_array_overtime(grid_change_over_time,2,rows_number*columns_number*step),color="#00FF00")
    plt.plot(target_status_array_overtime(grid_change_over_time,3,rows_number*columns_number*step),color="#FF0000")
    plt.plot(target_status_array_overtime(grid_change_over_time,4,rows_number*columns_number*step),color="#00FFF0")
    plt.plot(target_status_array_overtime(grid_change_over_time,5,rows_number*columns_number*step),color="#F0FF00")
    plt.xlabel("day")
    plt.ylabel("number of person")
    plt.title("")
    plt.legend(["susceptible", "exposed", "infected", "quarantined", "removed"])
    
if mode_choosing == 1:
    if t > rows_number*columns_number*step:
        print("The time you chose for x-axis was too long, choose a smaller number or change to the automatic mode. --LL")
    else:
        plt.figure(figsize=(x_size,y_size))
        plt.plot(target_status_array_overtime(grid_change_over_time,1,t),color="#001DFF")
        plt.plot(target_status_array_overtime(grid_change_over_time,2,t),color="#00FF00")
        plt.plot(target_status_array_overtime(grid_change_over_time,3,t),color="#FF0000")
        plt.plot(target_status_array_overtime(grid_change_over_time,4,t),color="#00FFF0")
        plt.plot(target_status_array_overtime(grid_change_over_time,5,t),color="#F0FF00")
        plt.xlabel("day")
        plt.ylabel("number of person")
        plt.title("")
        plt.legend(["susceptible", "exposed", "infected", "quarantined", "removed"])

model\_v.1.2.0 \-\- allows user to operate and compare maximal 4 different grids with different parametres at the same time



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def initial_grid_generator(n,m,D,I):
    
    y=np.zeros((n,m))
    
    for i in range (n):
        for j in range (m):
            if np.random.rand()<=D:
                if np.random.rand()<=I:
                    y[i,j] = infected
                else:
                    y[i,j] = susceptible
            else:
                y[i,j] = 0
                
    return y

status_array = [0,1,2,3,4,5]
empty_cell = 0
susceptible = S = 1
exposed = E = 2
infected = I = 3
quarantined = Q = 4
removed = R = 5

##############################################

# n,m determine the size of the grid, D is the population density over all cells, I is the infector density over population

n = 140
m = 140
x = 4

grid_all = np.zeros((n,m,x))
grid_all[:,:,0] = initial_grid_generator(n,m,0.6,0.001)
grid_all[:,:,1] = grid_all[:,:,0]
grid_all[:,:,2] = 0
grid_all[:,:,3] = 0

##############################################

cmap = mcolors.ListedColormap(["#000000", '#001DFF', "#00FF00", '#FF0000', "#00FFF0", '#F0FF00'])
bounds = [0, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

rows_number=1
columns_number=x
relative_size=5

fig, axes = plt.subplots(rows_number, columns_number, figsize=(columns_number*relative_size,rows_number*relative_size))

for xx in range (x):
    axes[xx].imshow(grid_all[:,:,xx], cmap= cmap, norm= norm)
    axes[xx].axis("off")


In [0]:
def grid_change_over_time(input_array,timesteps,xx):

    i_n,i_m=np.shape(input_array)
    o_n=i_n+2*d_max
    o_m=i_m+2*d_max
    operating_array= np.zeros((o_n, o_m, timesteps, 2))
    operating_array[d_max:d_max+i_n,d_max:d_max+i_m,0,0]=input_array[:,:]
    
    def neighbours_status_counter(input_array, i, j, target_status, d,t):
    
        y=np.sum((input_array[i-d:i+d+1,j-d:j+d+1,t,0] == target_status))-np.sum(input_array[i,j,t,0]==target_status)
        
        return y

    def cell_A_to_B_convertor(i,j,A,B):

        def p_s_e(i,j,sub_p_s_e,d):
            
            p_e= sub_p_s_e* (neighbours_status_counter(operating_array,i,j,2,d,t)+ neighbours_status_counter(operating_array,i,j,3,d,t))
            
            return p_e
        
        A_cell_at_t_plus_one=np.zeros(2)
        converting_probability_array=np.array([[0   ,0   ,0   ,0   ,0   ,0   ],
                                               [0   ,0   ,p_s_e(i,j,sub_p_s_e[xx],d[xx]),0   ,0   ,0   ],
                                               [0   ,0   ,0   ,p_e_i[xx] ,0   ,0   ],
                                               [0   ,0   ,0   ,0   ,p_i_q[xx] ,p_i_r[xx] ],
                                               [0   ,0   ,0   ,0   ,0   ,p_q_r[xx] ],
                                               [0   ,0   ,0   ,0   ,0   ,0   ]])
        transitional_delay_array=np.array([[0 ,0 ,0 ,0 ,0 ,0 ],
                                           [0 ,0 ,t_s_e[xx] ,0 ,0 ,0 ],
                                           [0 ,0 ,0 ,t_e_i[xx] ,0 ,0 ],
                                           [0 ,0 ,0 ,0 ,t_i_q[xx] ,t_i_r[xx] ],
                                           [0 ,0 ,0 ,0 ,0 ,t_q_r[xx] ],
                                           [0 ,0 ,0 ,0 ,0 ,0 ]])
                
        if operating_array[i,j,t,1] <= t-transitional_delay_array[A,B] and np.random.rand() <= converting_probability_array[A,B]:
            A_cell_at_t_plus_one[0] = B
            A_cell_at_t_plus_one[1] = t+1
        else:
            A_cell_at_t_plus_one[0] = A
            A_cell_at_t_plus_one[1] = operating_array[i,j,t,1]
        
        return A_cell_at_t_plus_one
    
    
    for t in range (timesteps-1):
        for i in range(d_max, i_n+d_max):
            for j in range(d_max, i_m+d_max):
                if operating_array[i,j,t,0] == 0:
                    operating_array[i,j,t+1,:] = 0
                if operating_array[i,j,t,0] == 1:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,1,2)
                if operating_array[i,j,t,0] == 2:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,2,3)
                if operating_array[i,j,t,0] == 3:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,5)
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,3,4)
                if operating_array[i,j,t,0] == 4:
                    operating_array[i,j,t+1,:] = cell_A_to_B_convertor(i,j,4,5)
                if operating_array[i,j,t,0] == 5:
                    operating_array[i,j,t+1,:] = operating_array[i,j,t,:]
                    
    result = operating_array[d_max:d_max+i_n, d_max:d_max+i_m, :, 0]
    
    return result

##############################################

d_max= 2
d= [2,2,2,2]

sub_p_s_e = [0.01,0.05,0.10,0.15]
p_e_i = [0.3,0.3,0.3,0.3]
p_i_q = [0.2,0.2,0.2,0.2]
p_i_r = [0.2,0.2,0.2,0.2]
p_q_r = [0.5,0.5,0.5,0.5]

t_s_e = [0,0,0,0]
t_e_i = [15,15,15,15]
t_i_q = [5,5,5,5]
t_i_r = [15,15,15,15]
t_q_r = [15,15,15,15]


rows_number=10
columns_number=1
step=15
relative_size=4

##############################################

# 0 turns on the graph generator
# 1 turns off the graph generator

graph_generator = 0

##############################################

timesteps=rows_number*columns_number*step
grid_change_over_time_all=np.zeros((n,m,timesteps,x))

for xx in range (x):
    
    grid_change_over_time_all[:,:,:,xx]=grid_change_over_time(grid_all[:,:,xx],timesteps,xx)
    
if graph_generator == 0:
    
    fig, axes = plt.subplots(rows_number, columns_number*x, figsize=(columns_number*relative_size*x,rows_number*relative_size))
    
    for xx in range (x):
        for j in range(columns_number):
            for i in range(rows_number):
                axes[i][columns_number*xx+j].imshow(grid_change_over_time_all[:,:,(i+rows_number*j)*step,xx], cmap= cmap, norm= norm)
                axes[i][columns_number*xx+j].axis("off")

    plt.show()
    
if graph_generator == 1:
    print("Mode choice: 1")

In [0]:
def cells_status_counter(input_array,target_status):
    
    y=np.sum(input_array[:,:] == target_status)
    
    return y

def target_status_array_overtime(grid_change_over_time,target_status,t):
    
    n,m,tt=np.shape(grid_change_over_time)
    result=np.zeros(t)
    
    for i in range (t):
        result[i]=cells_status_counter(grid_change_over_time[:,:,i],target_status)
        
    return result

##############################################

# 0 for auto mode, and the x axis would be the full size.
# 1 for manual mode, you have to choose the length of x-axis, sometimes it would help if you have got a too long timelength.

mode_choosing = 0
x_size = 6
y_size = 4
t= 100   # the length of the x-axis you are plotting(only work in manual mode)

##############################################

def curve_plotting(grid_change_over_time,t):
    plt.figure(figsize=(x_size,y_size))
    plt.plot(target_status_array_overtime(grid_change_over_time,1,t),color="#001DFF")
    plt.plot(target_status_array_overtime(grid_change_over_time,2,t),color="#00FF00")
    plt.plot(target_status_array_overtime(grid_change_over_time,3,t),color="#FF0000")
    plt.plot(target_status_array_overtime(grid_change_over_time,4,t),color="#00FFF0")
    plt.plot(target_status_array_overtime(grid_change_over_time,5,t),color="#F0FF00")
    plt.xlabel("day")
    plt.ylabel("number of person")
    plt.title("")
    plt.legend(["susceptible", "exposed", "infected", "quarantined", "removed"])
    
for xx in range (x):

    if mode_choosing == 0:
        curve_plotting(grid_change_over_time_all[:,:,:,xx],rows_number*columns_number*step)
    
    
    
    if mode_choosing == 1:
        if t > rows_number*columns_number*step:
            print("The time you chose for x-axis was too long, choose a smaller number or change to the automatic mode. --LL")
        else:
            curve_plotting(grid_change_over_time_all[:,:,:,xx],t)